<a href="https://colab.research.google.com/github/emm-gl/WA2501_Machine_Learning_AI/blob/main/MA2502_Matem%C3%A1ticasParaLaCienciaDeDatos/Examen_parcial_02_AlgebraLineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Examen parcial 02: Algebra Lineal

Emmanuel Gonzalez Calitl

Octubre 2025

Para aprobar el examen es necesario resolver los 3 ejercicios de la siguiente
lista. Debe entregarse un Google Collab y compartirse con los profesores del
curso sin darles permiso de edición. La fecha límite de entrega es el lunes de
la semana 9.



1. Sin utilizar Numpy defina una función en Python para multiplicar dos
matrices de tamaño arbitrario.


In [86]:
def MatMul(A, B):
    #Condiciones:
    #Revisar que el número de columnas de la matriz A sea igual al número de filas de la matriz B
    rowsA = len(A)
    colsB = len(B[0])

    if rowsA  != colsB:
      print("Error el número de filas de la matriz A debe ser igual al número de columnas de la matriz B")
      print("Número de filas de la matriz A: ", rowsA)
      print("Número de columnas de la matriz B: ", colsB)
      return

    #Multiplicación:
    C = [[0 for _ in range(colsB)] for _ in range(rowsA)]       #Initialize the new matrix

    for i in range(rowsA):                  #Iterar en cada fila de A
      for j in range(colsB):                #Iterar en cada columna B
        for k in range(len(B)):             #Iterar en cada fila de B
          C[i][j] += A[i][k] * B[k][j]      #Producto punto

    return C

In [88]:
#Matrices a multiplicar:
A = [
    [1, 2, 3],
    [4, 5, 6]
    ]

B = [[5, -1],
     [1, 0],
     [-2, 3]
     ]

C = MatMul(A, B)

#Impresión de Matriz de manera visual:
for fila in C:
    print(" ".join(map(str, fila)))

1 8
13 14



2. Utilizar la descomposición en valores singulares para resolver alguno
de los sistemas de ecuaciones de la primera semana.



3. Utilizando la base de datos MNIST, comparar la reducción de PCA contra la de descomposición en valores singulares. La métrica que utilizaremos será la medida del coseno promedio (al reducir la dimensión) entre las imágenes del mismo dígito.